In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
import numpy as np 
import pandas as pd
import os
import random
import json
from tqdm import tqdm
import re
import glob
from textblob import TextBlob
from functools import partial
import string
import nltk
import spacy
from nltk.probability import FreqDist

In [ ]:
RANDOM_SEED = 42
def seed_everything(seed=RANDOM_SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
seed_everything()

In [ ]:
train = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
train.head(3)

In [ ]:
for col in train.columns:
    print(col + ":" + str(len(train[col].unique())))

print("all rows = " + str(len(train)))

Data Description train.csv -labels and metadata for the training set train/tezt directory - the full text of the training/test set's publications in JSON format, broken into sections with section titles

id - publication id - note that there are multiple rows for some training documents, indicating multiple mentioned datasets. pub_title - title of the publication (a small number of publications have the same title). dataset_title - the title of the dataset that is mentioned within the publication. dataset_label - a portion of the text that indicates the dataset. cleaned_label - the dataset_label, as passed through the clean_text function from the Evaluation page. sample_submission.csv - a sample submission file in the correct format.

Id - publication id. PredictionString - To be filled with equivalent of cleaned_label of train data.


In [ ]:
sample = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/sample_submission.csv")
sample

In [ ]:
train.head(3)

In [ ]:
train_path = "../input/coleridgeinitiative-show-us-the-data/train"
test_path = "../input/coleridgeinitiative-show-us-the-data/test"

In [ ]:
all_train_path = [os.path.join(train_path,s) + ".json" for s in train["Id"]]
all_test_path = [os.path.join(test_path,s) + ".json" for s in sample["Id"]]

In [ ]:
json_path = all_train_path[0]
json_path

In [ ]:
with open(json_path, 'r') as f:
        json_decode = json.load(f)

In [ ]:
jsontest = pd.DataFrame(json_decode)
jsontest

In [ ]:
texts = ""

for a in jsontest.values:
    texts += a[0] +" "+ a[1] + " "

In [ ]:
texts[:300]

In [ ]:
json_path = all_train_path[0]
with open(json_path, 'r') as f:
        json_decode = json.load(f)

jsontest = pd.DataFrame(json_decode)

texts = ""

for a in jsontest.values:
    texts += a[0] +" "+ a[1] +" "

In [ ]:
%%time

alltexts = []

for json_path in tqdm(all_train_path):

    with open(json_path, 'r') as f:
            json_decode = json.load(f)
    jsontest = pd.DataFrame(json_decode)

    texts = ""

    for a in jsontest.values:
        texts += a[0] +" "+ a[1] + " "
        
    alltexts.append(texts)

In [ ]:
train["text"] = alltexts
train

In [ ]:
train.head(3)

In [ ]:
print(len(train["Id"].unique()))
print(len(train))

In [ ]:
Idgroup = train.groupby("Id")["dataset_label"].count().reset_index()
Idgroup.columns = ["Id","count"]
Idgroup = Idgroup.sort_values("count").reset_index(drop=True)
Idgroup

In [ ]:
mostId = train[train["Id"] == Idgroup["Id"].iloc[-1]]
mostId.head()

In [ ]:
mostIdlist = mostId["cleaned_label"].to_list()
mostIdlist

In [ ]:
mostIdlist = ("|").join(mostIdlist)
mostIdlist

In [ ]:
sample

In [ ]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

In [ ]:
check = []
for a in range(len(train)):
    
    if clean_text(train["dataset_label"].iloc[a]) == train["cleaned_label"].iloc[a]:
        check.append(1)
    else:
        check.append(0)

In [ ]:
np.sum(check)/len(train)

In [ ]:
train["check"] = check
checkdf = train[train["check"]==0]
checkdf.head(3)

In [ ]:
checkdf["dataset_label"].unique()

In [ ]:
checkdf["cleaned_label"].unique()

In [ ]:
clean_text(checkdf["dataset_label"].iloc[0])

In [ ]:
cleanlabel = []
for a in tqdm(train["cleaned_label"]):
    if a[-1] == " ":
        cleanlabel.append(a[:-1])
    else:
        cleanlabel.append(a)

In [ ]:
check = []
for a in range(len(train)):
    
    if clean_text(train["dataset_label"].iloc[a]) == cleanlabel[a]:
        check.append(1)
    else:
        check.append(0)

In [ ]:
np.sum(check)/len(train)

In [ ]:
train["cleaned_label"]=cleanlabel

In [ ]:
train.head(3)

In [ ]:
train["text"] = [clean_text(s) for s in tqdm(train["text"])]

In [ ]:
dslabel = [clean_text(s) for s in train["dataset_label"].unique()]
len(dslabel)

In [ ]:
label_control = []
all_labels = []
label_len = []

for a in tqdm(train["text"]):
    labels = []
    for b in dslabel:
        if b in a:
            labels.append(clean_text(b))
            break
    if len(labels)==0:
        label_control.append(0)
    else:
        label_control.append(1)
        
    #all_labels.append("|".join(labels))
    #label_len.append(len(labels))

In [ ]:
np.sum(label_control)/len(train)

In [ ]:
sample

In [ ]:
%%time

alltexts = []

for json_path in tqdm(all_test_path):

    with open(json_path, 'r') as f:
            json_decode = json.load(f)
    jsontest = pd.DataFrame(json_decode)

    texts = ""

    for a in jsontest.values:
        texts += a[0] + " " + a[1] + " "
        
    alltexts.append(texts)

In [ ]:
sample["text"] = alltexts

In [ ]:
sample

In [ ]:
sample["text"] = [clean_text(s) for s in tqdm(sample["text"])]

In [ ]:
print(len(dslabel))
dstitle = [clean_text(s) for s in train["dataset_title"].unique()]
dslabel = set(dslabel + dstitle) 
len(dslabel)

In [ ]:
label_control = []  
all_labels = []
label_len = []

for a in tqdm(sample["text"]):
    labels = []
    for b in dslabel:
        if b in a:
            labels.append(clean_text(b))
            
    if len(labels)==0:
        label_control.append(0)
    else:
        label_control.append(1)
    
    all_labels.append("|".join(labels))
    label_len.append(len(labels))

In [ ]:
sample["PredictionString"] = all_labels
sample

In [ ]:
sample["PredictionString"].iloc[2]

In [ ]:
sample = sample[["Id","PredictionString"]]

In [ ]:
sample.to_csv("submission.csv",index=False)

In [ ]:
sample

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))